In [2]:
API_KEY = "2RT8mQ01jQmMrZwiHczy75cckYTP2Kii9McSvgDCGFJ8Q7P37vVwJQQJ99BJACYeBjFXJ3w3AAAAACOGxEp9"

In [3]:
import os
from openai import AzureOpenAI

endpoint = "https://chr-umn-hackathon-resource.cognitiveservices.azure.com/"
model_name = "gpt-5-mini"
deployment = "gpt-5-mini"

subscription_key = "2RT8mQ01jQmMrZwiHczy75cckYTP2Kii9McSvgDCGFJ8Q7P37vVwJQQJ99BJACYeBjFXJ3w3AAAAACOGxEp9"
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": "I am going to Paris, what should I see?",
        }
    ],
    # max_completion_tokens=16384,
    model=deployment
)

print(response.choices[0].message.content)


Fantastic — Paris has a lot to offer. To make this useful quickly, here are (1) top highlights, (2) sample itineraries (1- and 3-day), (3) food & practical tips, and (4) a few safety/ticket notes. Tell me how many days you have and what you like (art, food, history, shopping, kids) and I’ll customize it.

Top sights (don’t miss)
- Eiffel Tower (book timed tickets; or enjoy the view from Trocadéro or Champ de Mars)
- Louvre Museum (book ahead; closed Tuesdays)
- Musée d’Orsay (Impressionists; closed Mondays)
- Notre‑Dame area & Île de la Cité (cathedral restoration ongoing, but the island and Sainte‑Chapelle are beautiful)
- Sainte‑Chapelle (stunning stained glass)
- Montmartre & Sacré‑Cœur (great views, artists’ square)
- Arc de Triomphe & Champs‑Élysées
- Le Marais (Place des Vosges, boutiques, Jewish quarter)
- Luxembourg Gardens & Latin Quarter
- Seine cruise (Bateaux Mouches) — lovely at dusk
- Pompidou Centre (modern art and great view)
- Musée Rodin / Musée de l’Orangerie (Monet’

In [4]:
import os
from openai import AzureOpenAI

from typing import Annotated

from langchain.chat_models import init_chat_model
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

os.environ["AZURE_OPENAI_API_KEY"] = API_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://chr-umn-hackathon-resource.cognitiveservices.azure.com/"
os.environ["OPENAI_API_VERSION"] = "2025-03-01-preview"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "gpt-5-mini"

llm = init_chat_model(
    "azure_openai:gpt-5-mini",
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
)


def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


# The first argument is the unique node name
# The second argument is the function or object that will be called whenever
# the node is used.
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)
graph = graph_builder.compile()

In [5]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    print("No")
    pass

No


In [8]:

messages = [
    {"role": "system", "content": "You are a helpful assistant that handles negotiating the rates to haul a load"},
    {"role": "system", "content": '''The user should tell you the load which includes: departure place and destination like (Seattle, WA, Denver, CO);
    Commodity and its special needs (like reefer required), weight, distance, and target price. If they don't provide this, you should ask them.'''},
    {"role": "system", "content": "Please talk like a human, ask one requirement at a time"},
    {"role": "system", "content": "If the target price is too low, negotiate with the customers"}
]

def stream_graph_updates(user_input: str):
    print("User:", user_input)
    messages.append({"role": "user", "content": user_input})
    for event in graph.stream({"messages": messages}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)


while True:
    try:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break
        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "What do you know about LangGraph?"
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

User: I want to load from Memphis, TN → Miami, FL
Assistant: Got it — Memphis, TN to Miami, FL. What commodity are you hauling, and does it have any special requirements (reefer, hazmat, liftgate, palletized, etc.)?
User: Consumer packaged goods (dry van)
Assistant: Got it — Memphis, TN → Miami, FL, CPG on a dry van. What’s the weight (or pallet count) of the load?
User: 44,000 lbs
Assistant: Thanks — got Memphis, TN → Miami, FL, consumer packaged goods (dry van), 44,000 lbs. What’s your target price for this load — total or per mile?
User: $1,700
Assistant: Do you know the total loaded miles for Memphis, TN → Miami, FL, or should I estimate the distance for you?
User: 1,050 miles
Assistant: Got it — Memphis, TN → Miami, FL; CPG on a dry van; 44,000 lbs; 1,050 miles; target $1,700.

Quick question: when’s the pickup date/time?

Heads-up: $1,700 is below market for that lane and weight (that’s about $1.62/mi). Real costs (fuel, driver pay, tolls, equipment and a needed margin) put a rea